In [2]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from mlserve_sdk.client import MLServeClient
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

True

In [16]:
# -----------------------------
# 1. Connect to MLServe
# -----------------------------
USERNAME = os.getenv("USERNAME")
TOKEN = os.getenv("TOKEN")

client = MLServeClient()
client.login(USERNAME, TOKEN)

In [7]:
# -----------------------------
# 2. Train a sample model
# -----------------------------
X, y = load_iris(return_X_y=True)
model = LogisticRegression(max_iter=200)
model.fit(X, y)
score=model.score(X, y)

In [8]:
# -----------------------------
# 3. Deploy model
# -----------------------------
feats=["sepal_length", "sepal_width", "petal_length", "petal_width"]
client.deploy(
    model=model,
    name="iris",
    version="v1",
    features=feats,
    background_df=pd.DataFrame(X, columns=feats).sample(100).reset_index(drop=True),
    task_type='classification',
    metrics={'accuracy':score}
)

{'predict_url': 'https://mlserve.com/api/v1/predict/iris/v1'}

In [32]:
# -----------------------------
# 4. Predict (specific version)
# -----------------------------
iris_data = {
    "inputs": [
        [5.1, 3.5, 1.4, 0.2],  # Iris-setosa
    ]
}
preds = client.predict("iris", "v1", iris_data)
print("Predictions:", preds)

Predictions: {'version': 'v1', 'prediction_ids': ['9a300880-455d-4dd5-abea-2937810fa1db'], 'predictions': [0], 'explanations': [[]]}


In [33]:
# -----------------------------
# 7. Fetch Basic Metrics
# -----------------------------
metrics = client.get_metrics("iris", "v1", as_dataframe=True)
metrics

,requests,predictions,throughput_rps,prediction_rps,avg_latency_ms,p50_latency_ms,p95_latency_ms,p99_latency_ms,avg_latency_per_element_ms,p50_latency_per_element_ms,p95_latency_per_element_ms,p99_latency_per_element_ms,error_rate
timestamp,,,,,,,,,,,,,
2025-09-17 18:00:00,2,2,0.000556,0.000556,77.413091,77.413091,128.997065,133.582307,77.413091,77.413091,128.997065,133.582307,0.0
2025-09-17 20:00:00,13,13,0.003611,0.003611,20.230519,17.456689,32.839212,44.536259,20.230519,17.456689,32.839212,44.536259,0.0
